In [ ]:
!pip install openai-agents
!pip install python-dotenv
!pip install gradio

In [ ]:
from __future__ import annotations as _annotations

import nest_asyncio
import random
import uuid
import asyncio
from pydantic import BaseModel
import os
from dotenv import load_dotenv
import gradio as gr

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

# Load environment variables (assuming API key is set in Colab secrets or hardcoded for simplicity)
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY') or "your-openai-api-key-here"  # Replace with your key or use Colab secrets
os.environ['OPENAI_API_KEY'] = api_key

# Import agent-related modules (assuming these are available in your environment)
from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents.tracing import set_tracing_export_api_key

# Set tracing API key
set_tracing_export_api_key(api_key)

# Define BusinessAdvisorContext
class BusinessAdvisorContext(BaseModel):
    business_owner_name: str | None = None
    business_type: str | None = None
    business_stage: str | None = None
    business_id: str | None = None

# Define Tools
@function_tool(
    name_override="business_faq_lookup", description_override="Lookup frequently asked business questions."
)
async def business_faq_lookup(question: str) -> str:
    if "register" in question or "license" in question:
        return (
            "To register a business, you typically need to: \n"
            "1. Choose a business structure (LLC, Sole Proprietorship, etc.)\n"
            "2. Register with your state's business division\n"
            "3. Obtain an EIN from the IRS\n"
            "4. Apply for necessary licenses and permits"
        )
    elif "funding" in question or "loan" in question:
        return (
            "Small business funding options include:\n"
            "- SBA loans (3-5% interest rates, 5-25 year terms)\n"
            "- Traditional bank loans\n"
            "- Business credit cards\n"
            "- Angel investors or venture capital\n"
            "- Crowdfunding platforms\n"
            "- Grants for specific industries or demographics"
        )
    elif "marketing" in question or "advertise" in question:
        return (
            "Cost-effective marketing strategies for small businesses:\n"
            "- Social media marketing\n"
            "- Email marketing campaigns\n"
            "- Content marketing and SEO\n"
            "- Local partnerships and networking\n"
            "- Customer referral programs"
        )
    elif "tax" in question:
        return (
            "Small businesses typically need to pay:\n"
            "- Income tax (federal, state, local)\n"
            "- Self-employment tax (if applicable)\n"
            "- Employment taxes (if you have employees)\n"
            "- Sales tax (depending on your products/services)\n"
            "- Property tax (if you own business property)\n\n"
            "Consult with a tax professional for specific advice."
        )
    return "I'm sorry, I don't have specific information on that business topic. I'll connect you with our general business advisor for more help."

@function_tool
async def create_business_plan(
    context: RunContextWrapper[BusinessAdvisorContext],
    business_type: str,
    target_market: str,
    startup_costs: str,
    revenue_model: str
) -> str:
    context.context.business_type = business_type
    context.context.business_stage = "planning"
    assert context.context.business_id is not None, "Business ID is required"
    business_plan = (
        f"\nBasic Business Plan Outline for {business_type} Business:\n\n"
        f"1. Executive Summary\n"
        f"   - Business Type: {business_type}\n"
        f"   - Target Market: {target_market}\n"
        f"   - Initial Investment: {startup_costs}\n"
        f"   - Revenue Model: {revenue_model}\n\n"
        f"2. Market Analysis\n"
        f"   - Recommended research: Conduct surveys with potential customers in the {target_market} demographic\n"
        f"   - Competition analysis: Identify 3-5 main competitors in the {business_type} space\n\n"
        f"3. Financial Projections\n"
        f"   - Initial Investment: {startup_costs}\n"
        f"   - Projected break-even: Typically 12-18 months for a {business_type} business\n"
        f"   - Revenue streams: {revenue_model}\n\n"
        f"4. Next Steps\n"
        f"   - Finalize market research\n"
        f"   - Secure funding ({startup_costs})\n"
        f"   - Develop detailed marketing plan for {target_market}\n"
        f"   - Create legal structure and register business\n\n"
        f"Business plan ID: {context.context.business_id}"
    )
    return business_plan

@function_tool
async def recommend_marketing_strategy(
    context: RunContextWrapper[BusinessAdvisorContext],
    business_type: str,
    target_audience: str,
    monthly_budget: str
) -> str:
    context.context.business_type = business_type
    budget_amount = float(''.join(c for c in monthly_budget if c.isdigit() or c == '.') or '0')
    low_budget = budget_amount < 500
    medium_budget = 500 <= budget_amount < 2000
    high_budget = budget_amount >= 2000
    strategies = [
        "- Social media presence (organic posting)",
        "- Email marketing to existing customers",
        "- Local networking and partnerships"
    ]
    if medium_budget or high_budget:
        strategies.extend(["- Targeted social media advertising", "- Basic SEO optimization", "- Content marketing (blog/video)"])
    if high_budget:
        strategies.extend(["- Professional website redesign", "- Comprehensive SEO campaign", "- Influencer partnerships", "- Local radio/print advertising"])
    if "retail" in business_type.lower():
        strategies.append("- In-store promotions and loyalty program")
    elif "restaurant" in business_type.lower():
        strategies.append("- Food delivery platform partnerships")
    elif "service" in business_type.lower():
        strategies.append("- Customer testimonial campaign")
    budget_level = "low" if low_budget else "medium" if medium_budget else "high"
    return (
        f"\nMarketing Strategy Recommendations for {business_type} targeting {target_audience}\n"
        f"Budget Level: {budget_level.upper()} (${budget_amount:.2f}/month)\n\n"
        f"Recommended Strategies:\n" + "\n".join(strategies) + "\n\n"
        f"Implementation Timeline:\n"
        f"Month 1: Set up social channels, create content calendar\n"
        f"Month 2: Implement customer email collection, begin regular posting\n"
        f"Month 3: Analyze results, adjust strategy, expand to additional channels\n\n"
        f"Expected outcomes:\n"
        f"- Increased brand awareness within 1-2 months\n"
        f"- First customers from marketing efforts within 2-3 months\n"
        f"- Positive ROI on marketing spend within 4-6 months"
    )

@function_tool
async def funding_calculator(
    context: RunContextWrapper[BusinessAdvisorContext],
    startup_costs: str,
    monthly_expenses: str,
    expected_revenue_timeline: str
) -> str:
    try:
        startup_costs_amount = float(''.join(c for c in startup_costs if c.isdigit() or c == '.'))
        monthly_expenses_amount = float(''.join(c for c in monthly_expenses if c.isdigit() or c == '.'))
    except ValueError:
        return "Please provide numeric values for startup costs and monthly expenses."
    months = int(''.join(c for c in expected_revenue_timeline if c.isdigit())) if "month" in expected_revenue_timeline.lower() else 6
    runway_costs = monthly_expenses_amount * months
    total_funding_needed = startup_costs_amount + runway_costs
    funding_options = []
    if total_funding_needed < 10000:
        funding_options.extend(["- Personal savings or friends and family", "- Business credit cards", "- Microloans from community organizations"])
    if 5000 <= total_funding_needed < 100000:
        funding_options.extend(["- SBA Microloan Program (up to $50,000)", "- Online lenders like Kabbage or OnDeck", "- Crowdfunding platforms like Kickstarter or Indiegogo"])
    if total_funding_needed >= 50000:
        funding_options.extend(["- SBA 7(a) loans (up to $5 million)", "- Traditional bank business loans", "- Angel investors (typically for scalable businesses)"])
    funding_options.append("- Small business grants (industry or demographic specific)")
    return (
        f"\nFunding Analysis:\n\n"
        f"Initial startup costs: ${startup_costs_amount:,.2f}\n"
        f"Monthly expenses: ${monthly_expenses_amount:,.2f}\n"
        f"Expected time until revenue: {months} months\n"
        f"Recommended runway buffer: {months} months\n\n"
        f"Total estimated funding needed: ${total_funding_needed:,.2f}\n"
        f"(${startup_costs_amount:,.2f} startup + ${runway_costs:,.2f} operating expenses)\n\n"
        f"Recommended funding options:\n" + "\n".join(funding_options) + "\n\n"
        f"Funding preparation steps:\n"
        f"1. Create detailed business plan with financial projections\n"
        f"2. Prepare personal financial statements\n"
        f"3. Check and improve credit score if needed\n"
        f"4. Gather documentation (business registration, tax returns, etc.)\n"
        f"5. Research specific requirements for each funding option"
    )

# Define Hooks
async def on_startup_handoff(context: RunContextWrapper[BusinessAdvisorContext]) -> None:
    business_id = f"BIZ-{random.randint(1000, 9999)}"
    context.context.business_id = business_id

async def on_marketing_handoff(context: RunContextWrapper[BusinessAdvisorContext]) -> None:
    if not context.context.business_id:
        business_id = f"BIZ-{random.randint(1000, 9999)}"
        context.context.business_id = business_id

async def on_funding_handoff(context: RunContextWrapper[BusinessAdvisorContext]) -> None:
    if not context.context.business_id:
        business_id = f"BIZ-{random.randint(1000, 9999)}"
        context.context.business_id = business_id

# Define Agents
business_faq_agent = Agent[BusinessAdvisorContext](
    name="Business FAQ Agent",
    handoff_description="A helpful agent that can answer common questions about starting and running a small business.",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX}\nYou are a Business FAQ Agent. If you are speaking to a business owner, you were likely transferred from the triage agent.\nUse the following routine to support the business owner.\n# Routine\n1. Identify the specific business question being asked.\n2. Use the business FAQ lookup tool to answer the question. Stick to the information provided by the tool.\n3. If you cannot answer the question with the available information, transfer back to the triage agent.",
    tools=[business_faq_lookup],
)

startup_guide_agent = Agent[BusinessAdvisorContext](
    name="Startup Guide Agent",
    handoff_description="A helpful agent that assists with business formation and creating initial business plans.",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX}\nYou are a Startup Guide Agent. If you are speaking to a business owner, you were likely transferred from the triage agent.\nUse the following routine to support the business owner.\n# Routine\n1. Ask about their business idea, including:\n   - What type of business they want to start\n   - Who their target market is\n   - Their estimated startup costs\n   - How they plan to make money (revenue model)\n2. Use the create business plan tool to generate a basic business plan outline.\n3. Review the plan with the client and suggest next steps.\nIf the business owner asks a question that is not related to business formation or planning, transfer back to the triage agent.",
    tools=[create_business_plan],
)

marketing_agent = Agent[BusinessAdvisorContext](
    name="Marketing Agent",
    handoff_description="A helpful agent that provides marketing advice and strategy tips for small businesses.",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX}\nYou are a Marketing Agent. If you are speaking to a business owner, you were likely transferred from the triage agent.\nUse the following routine to support the business owner.\n# Routine\n1. Ask about their marketing needs, including:\n   - Their type of business\n   - Their target audience\n   - Their monthly marketing budget\n2. Use the recommend marketing strategy tool to generate personalized marketing recommendations.\n3. Explain the recommendations and answer any follow-up questions about implementing them.\nIf the business owner asks a question that is not related to marketing or customer acquisition, transfer back to the triage agent.",
    tools=[recommend_marketing_strategy],
)

funding_agent = Agent[BusinessAdvisorContext](
    name="Funding Agent",
    handoff_description="A helpful agent that assists with understanding financing options for small businesses.",
    instructions=f"{RECOMMENDED_PROMPT_PREFIX}\nYou are a Funding Agent. If you are speaking to a business owner, you were likely transferred from the triage agent.\nUse the following routine to support the business owner.\n# Routine\n1. Ask about their funding needs, including:\n   - Their initial startup costs\n   - Their expected monthly expenses\n   - When they expect to start generating revenue\n2. Use the funding calculator tool to analyze their needs and recommend appropriate funding options.\n3. Explain the funding options and answer any follow-up questions about applying for funding.\nIf the business owner asks a question that is not related to funding or finances, transfer back to the triage agent.",
    tools=[funding_calculator],
)

triage_agent = Agent[BusinessAdvisorContext](
    name="Business Advisor Triage",
    handoff_description="A triage agent that can direct business owners to specialized business advisors.",
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} "
        "You are a helpful Business Advisor Triage agent. Your job is to understand the business owner's needs "
        "and direct them to the appropriate specialized agent. You should get the business owner's name if possible."
        "\n\n"
        "If they're asking about business registration, licenses, taxes, marketing strategies, or general business questions, "
        "direct them to the Business FAQ Agent."
        "\n\n"
        "If they're interested in starting a new business, need help with a business plan, or are in the early "
        "formation stages, direct them to the Startup Guide Agent."
        "\n\n"
        "If they need help with marketing, customer acquisition, or advertising strategies, direct them to the "
        "Marketing Agent."
        "\n\n"
        "If they're looking for funding options, loans, or investment guidance, direct them to the Funding Agent."
    ),
    handoffs=[
        business_faq_agent,
        handoff(agent=startup_guide_agent, on_handoff=on_startup_handoff),
        handoff(agent=marketing_agent, on_handoff=on_marketing_handoff),
        handoff(agent=funding_agent, on_handoff=on_funding_handoff),
    ],
)

# Add handoff back to triage for all specialized agents
business_faq_agent.handoffs.append(triage_agent)
startup_guide_agent.handoffs.append(triage_agent)
marketing_agent.handoffs.append(triage_agent)
funding_agent.handoffs.append(triage_agent)

# Define Conversation State
class ConversationState:
    def __init__(self):
        self.current_agent = triage_agent
        self.input_items = []
        self.context = BusinessAdvisorContext()
        self.conversation_id = uuid.uuid4().hex[:16]

# Initialize global state
state = ConversationState()

# Define the async function for Gradio
async def business_advisor(message, history):
    global state
    state.input_items.append({"content": message, "role": "user"})
    # Process with some agent or logic
    result = await Runner.run(state.current_agent, state.input_items, context=state.context)
    response = ""
    for new_item in result.new_items:
        if isinstance(new_item, MessageOutputItem):
            response += f"{new_item.agent.name}: {ItemHelpers.text_message_output(new_item)}\n"
    state.input_items = result.to_input_list()
    return response

# Create and launch Gradio interface
demo = gr.ChatInterface(
    fn=business_advisor,
    title="Small Business Advisor",
    description="Ask me anything about starting or running a small business!"
)
demo.launch(share=True, debug=True)  # share=True provides a public URL in Colab

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d62a3ee1242ca8996c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://d62a3ee1242ca8996c.gradio.live
